In [1]:
import tensorflow as tf
import tensorflow.nn.rnn_cell as rnn
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.initializers import Constant
import tensorflow.keras.backend as K

In [2]:
from matplotlib.pyplot import plot
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
wts=np.load('weight6.npz')
weights=wts[wts.files[0]]
re_train=True
time_steps=1
n_input =6
num_units=64
n_classes=1
lr =0.0001

In [4]:
data_x=np.loadtxt('./TapData/data_raw_active.csv',delimiter=",")
data_y=np.loadtxt('./TapData/data_y_active.csv',delimiter=",")
preY = np.loadtxt('./TapData/data_preY_3C_fullData.active.csv',delimiter=",")
datay=data_y
data_y=data_y.reshape(len(data_y),1)
data_x=data_x[:,(6-n_input):6].reshape((len(data_x),n_input))
data_state = np.zeros([1,num_units*2],dtype='float')

loss_weight = preY*len(preY)/preY.sum()+datay*len(datay)/datay.sum()+1
loss_weight = loss_weight.reshape([len(loss_weight),1])

In [5]:
x= tf.placeholder('float',[None,n_input],name='input_tensor')
y = tf.placeholder('float',[None,n_classes],name='labels')
#inputs = tf.unstack(x,time_steps,1,name='input_tensor')
#state_h = tf.placeholder('float',[None,num_units],name='input_h')
#state_c = tf.placeholder('float',[None,num_units],name='input_c')
state_in   = tf.placeholder('float',[None,num_units*2],name='input_state')
loss_weight_in = tf.placeholder('float',[None,1],name='loss_weight')


In [6]:
'''
wi=tf.Variable(name='wi',initial_value=(weights[0][:,0:num_units]))
wf=tf.Variable(name='wf',initial_value=(weights[0][:,num_units:num_units*2]))
wc=tf.Variable(name='wc',initial_value=(weights[0][:,num_units*2:num_units*3]))
wo=tf.Variable(name='wo',initial_value=(weights[0][:,num_units*3:num_units*4]))
ui=tf.Variable(name='ui',initial_value=(weights[1][:,0:num_units]))
uf=tf.Variable(name='uf',initial_value=(weights[1][:,num_units:num_units*2]))
uc=tf.Variable(name='uc',initial_value=(weights[1][:,num_units*2:num_units*3]))
uo=tf.Variable(name='uo',initial_value=(weights[1][:,num_units*3:num_units*4]))

bi=tf.Variable(name='bi',initial_value=(weights[2][0:num_units]))
bf=tf.Variable(name='bf',initial_value=(weights[2][num_units:num_units*2]))
#bf=tf.Variable(name='bf',initial_value=(np.ones(num_units)))
bc=tf.Variable(name='bc',initial_value=(weights[2][num_units*2:num_units*3]))
bo=tf.Variable(name='bo',initial_value=(weights[2][num_units*3:num_units*4]))


out_weights=tf.Variable(name="weights",initial_value=weights[3])
out_bias=tf.Variable(name="bias",initial_value=weights[4])
'''

wi=tf.Variable(name='wi',initial_value=tf.random_normal([n_input, num_units], stddev=0.35))
wf=tf.Variable(name='wf',initial_value=tf.random_normal([n_input, num_units], stddev=0.35))
wc=tf.Variable(name='wc',initial_value=tf.random_normal([n_input, num_units], stddev=0.35))
wo=tf.Variable(name='wo',initial_value=tf.random_normal([n_input, num_units], stddev=0.35))
ui=tf.Variable(name='ui',initial_value=tf.random_normal([num_units, num_units], stddev=0.35))
uf=tf.Variable(name='uf',initial_value=tf.random_normal([num_units, num_units], stddev=0.35))
uc=tf.Variable(name='uc',initial_value=tf.random_normal([num_units, num_units], stddev=0.35))
uo=tf.Variable(name='uo',initial_value=tf.random_normal([num_units, num_units], stddev=0.35))

bi=tf.Variable(name='bi',initial_value=tf.random_normal([num_units], stddev=0.35))
bf=tf.Variable(name='bf',initial_value=tf.random_normal([num_units], stddev=0.35))
bc=tf.Variable(name='bc',initial_value=tf.random_normal([num_units], stddev=0.35))
bo=tf.Variable(name='bo',initial_value=tf.random_normal([num_units], stddev=0.35))

out_weights=tf.Variable(name="weights",initial_value=tf.random_normal([num_units, 1], stddev=0.35))
out_bias=tf.Variable(name="bias",initial_value=tf.random_normal([1], stddev=0.35))

'''
out_weights1=tf.Variable(name="weights1",initial_value=tf.random_normal([num_units, 16], stddev=0.35))
out_weights2=tf.Variable(name="weights2",initial_value=tf.random_normal([16, 1], stddev=0.35))
out_bias1=tf.Variable(name="bias",initial_value=tf.random_normal([16], stddev=0.35))
out_bias2=tf.Variable(name="bias",initial_value=tf.random_normal([1], stddev=0.35))
'''



Instructions for updating:
Colocations handled automatically by placer.


'\nout_weights1=tf.Variable(name="weights1",initial_value=tf.random_normal([num_units, 16], stddev=0.35))\nout_weights2=tf.Variable(name="weights2",initial_value=tf.random_normal([16, 1], stddev=0.35))\nout_bias1=tf.Variable(name="bias",initial_value=tf.random_normal([16], stddev=0.35))\nout_bias2=tf.Variable(name="bias",initial_value=tf.random_normal([1], stddev=0.35))\n'

In [7]:
state_h,state_c = tf.split(state_in,num_or_size_splits=2,axis=1)
x_i = K.dot(x,wi)
x_f = K.dot(x,wf)
x_c = K.dot(x,wc)
x_o = K.dot(x,wo)

i = K.hard_sigmoid(tf.add(tf.add(x_i,bi),K.dot(state_h,ui)))
f = K.hard_sigmoid(tf.add(tf.add(x_f,bf),K.dot(state_h,uf)))
c = tf.add(tf.multiply(f,state_c),K.tanh(tf.add(tf.add(x_c,bc),K.dot(state_h,uc))),name='output_c')
o = K.hard_sigmoid(tf.add(tf.add(x_o,bo),K.dot(state_h,uo)))
h = tf.multiply(o,tf.tanh(c),name='output_h')

prediction=tf.nn.relu(tf.add(tf.matmul(h,out_weights), out_bias,name='output_forRelu'), name="output")
state_out = tf.concat([h,c],axis=1,name='output_state')
prediction_weight=tf.matmul(prediction,loss_weight_in)
y_weight=tf.matmul(y,loss_weight_in)


In [8]:
loss=tf.losses.mean_squared_error(predictions=prediction_weight,labels=y_weight)
opt=tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)


Instructions for updating:
Use tf.cast instead.


In [9]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [11]:

tf.train.Saver()
iters=1
predictY=np.zeros([len(data_x)])
while iters<100:
    for i in tqdm(range(len(data_x))):
        feed={x:data_x[i:i+1],
              state_in:data_state,
                y:data_y[i:i+1],
              loss_weight_in:loss_weight[i:i+1]
               }
        if re_train:
            sess.run(opt,feed_dict=feed)

        data_state,_predictY = sess.run([state_out,prediction],feed_dict=feed)
        #print(data_h.shape)
        #data_c = sess.run(c,feed_dict=feed)
        predictY[i]=_predictY[0]
        #print(i)
    if iters%1==0:
        print(iters)
        mse = (np.square(predictY-data_y.reshape([len(data_x)]))).mean(axis=0)
        loss_weight = loss_weight.reshape([len(loss_weight)])
        mse_weight =np.square(np. multiply((predictY-data_y.reshape([len(data_x)])),loss_weight)).mean(axis=0)
        loss_weight = loss_weight.reshape([len(loss_weight),1])
        print('mse:'+str(mse))
        print('mse_weight:'+str(mse_weight))

        if iters%10==0:

            loss_weight = predictY*len(predictY)/predictY.sum()+datay*len(datay)/datay.sum()+1
            loss_weight = loss_weight.reshape([len(loss_weight),1])



    output_graph='./tmp/model/real_LSTM_fullData_dynamicW_tfTrain_7_8_6c_'+str(iters)+'.pb'
    output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess, # The session is used to retrieve the weights
            tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes 
            ['output','output_state'] # The output node names are used to select the usefull nodes
        ) 
    with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
    iters=iters+1
    np.savetxt('TF_predictY6C.csv',predictY)

100%|██████████████████████████████████████████████████████████████████████████| 96379/96379 [01:28<00:00, 1091.24it/s]


1
mse:0.045455006481545365
mse_weight:4.06455510025957
INFO:tensorflow:Froze 11 variables.
INFO:tensorflow:Converted 11 variables to const ops.


100%|██████████████████████████████████████████████████████████████████████████| 96379/96379 [01:26<00:00, 1115.92it/s]


2
mse:0.04080010522720412
mse_weight:3.9113097451354295
INFO:tensorflow:Froze 11 variables.
INFO:tensorflow:Converted 11 variables to const ops.


100%|██████████████████████████████████████████████████████████████████████████| 96379/96379 [01:29<00:00, 1076.74it/s]


...

97
mse:0.03796216902063532
mse_weight:2.060207684544059
INFO:tensorflow:Froze 11 variables.
INFO:tensorflow:Converted 11 variables to const ops.


100%|██████████████████████████████████████████████████████████████████████████| 96379/96379 [01:29<00:00, 1082.32it/s]


98
mse:0.04831064205299673
mse_weight:2.1257434533222086
INFO:tensorflow:Froze 11 variables.
INFO:tensorflow:Converted 11 variables to const ops.


100%|██████████████████████████████████████████████████████████████████████████| 96379/96379 [01:27<00:00, 1104.92it/s]


99
mse:0.04924369770756093
mse_weight:2.1898392963299913
INFO:tensorflow:Froze 11 variables.
INFO:tensorflow:Converted 11 variables to const ops.


In [9]:

#saved_model_dir='realtime_LSTM_32U_3C.hdf5'
#saved_model_dir='myModel.h5'
#frozen_graph = 'realtime_LSTM_v2_Tap_32_3Cepoch_450loss_0.08.h5.pb'
#converter=tf.lite.TFLiteConverter.from_keras_model_file(saved_model_dir)
iters=99
output_graph='./tmp/model/real_LSTM_fullData_dynamicW_tfTrain_7_8_6c_'+str(iters)+'.pb'
converter=tf.lite.TFLiteConverter.from_frozen_graph(output_graph,input_arrays={'input_tensor','input_state'},
                                                    output_arrays={'output','output_state'})
tflite_model = converter.convert()
open("./tmp/model/realLSTM32U6C_fullData_DynamicWpreY_7_08.tflite", "wb").write(tflite_model)

20468

In [13]:
np.savetxt('TF_predictY.csv',predictY)

In [14]:
import json
with open ('testTap.txt') as f:
    data = json.load(f)

valid_y=np.empty((0,1))
valid_x = np.empty((0,3))
for i in range(len(data)):
    p=data[i]['p']
    d=data[i]['d']
    for j in range(len(p)):
        dy=float(p[j])
        valid_y=np.append(valid_y,np.array(dy).reshape([1,1]),axis=0)
        datax = float(d[j][0]['x'])
        datay = float(d[j][0]['y'])
        dataz = float(d[j][0]['z'])
        valid_x = np.append(valid_x,np.array([datax,datay,dataz]).reshape([1,3]),axis=0)

In [28]:
type(data_x[0][0])

numpy.float64

In [10]:
predict_validY=np.zeros(len(valid_y))


In [13]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    for i in range(len(valid_x)):
        feed={x:valid_x[i:i+1],
              state_in:data_state,
                y:valid_y[i:i+1]
               }
        #sess.run(opt,feed_dict=feed)

        data_state,_predictY = sess.run([state_out,prediction],feed_dict=feed)
        #print(data_h.shape)
        #data_c = sess.run(c,feed_dict=feed)
        predict_validY[i]=_predictY[0]

TypeError: Cannot interpret feed_dict key as Tensor: Can not convert a float into a Tensor.

In [32]:
mse = (np.square(predictY-data_y.reshape([len(data_x)]))).mean(axis=0)

In [11]:
[n.name for n in tf.get_default_graph().as_graph_def().node]


['input_tensor',
 'labels',
 'input_state',
 'wi/initial_value',
 'wi',
 'wi/Assign',
 'wi/read',
 'wf/initial_value',
 'wf',
 'wf/Assign',
 'wf/read',
 'wc/initial_value',
 'wc',
 'wc/Assign',
 'wc/read',
 'wo/initial_value',
 'wo',
 'wo/Assign',
 'wo/read',
 'ui/initial_value',
 'ui',
 'ui/Assign',
 'ui/read',
 'uf/initial_value',
 'uf',
 'uf/Assign',
 'uf/read',
 'uc/initial_value',
 'uc',
 'uc/Assign',
 'uc/read',
 'uo/initial_value',
 'uo',
 'uo/Assign',
 'uo/read',
 'bi/initial_value',
 'bi',
 'bi/Assign',
 'bi/read',
 'bf/initial_value',
 'bf',
 'bf/Assign',
 'bf/read',
 'bc/initial_value',
 'bc',
 'bc/Assign',
 'bc/read',
 'bo/initial_value',
 'bo',
 'bo/Assign',
 'bo/read',
 'weights/initial_value',
 'weights',
 'weights/Assign',
 'weights/read',
 'bias/initial_value',
 'bias',
 'bias/Assign',
 'bias/read',
 'Const',
 'split/split_dim',
 'split',
 'MatMul',
 'MatMul_1',
 'MatMul_2',
 'MatMul_3',
 'Add',
 'MatMul_4',
 'Add_1',
 'Sigmoid',
 'Add_2',
 'MatMul_5',
 'Add_3',
 'Sigm

In [43]:
with tf.Session() as sess:
    sess.run(init)
    feed={x:data_x[1:1+1],
                  state_in:data_state,
                    y:data_y[1:1+1]
                   }
    Testa,Testb,Testc =sess.run([h,state_out,state_h],feed_dict=feed)
    print(Testa)

[[-0.24886423  0.32875472  0.78403926  0.31022385 -0.08955738 -0.2387976
   0.1099491  -1.          0.955397    0.06148118 -0.3927346   0.9998962
  -0.7832995  -0.2448097   0.06613222  0.12416882 -0.907323    0.3785977
   0.11073031  0.1624959  -0.6125419  -0.54658186 -0.47409666  0.33459553
  -0.1599297  -0.07685284  0.09279587 -0.40204054  0.24354106 -0.75412846
   0.4073003  -0.7056438 ]]


In [37]:
Testb

array([[-0.26087013, -0.07923192,  0.10833303, -0.08073159,  0.02192846,
        -0.09816513, -0.00919417, -0.16754107, -0.270551  , -0.14367388,
         0.22230028,  0.35295212, -0.39103547, -0.3484728 , -0.33515686,
        -0.0527762 , -0.5351516 ,  0.16447161,  0.07066188,  0.07050962,
        -0.19926658, -0.24653296, -0.21245113, -0.0577476 ,  0.11635146,
        -0.01639744, -0.14043774,  0.01351559,  0.17662126, -0.24192184,
         0.21953566, -0.4643797 , -0.61122173, -0.16257624,  0.19756094,
        -0.16770077,  0.04535002, -0.22916265, -0.02111781, -0.28886867,
        -0.50992864, -0.25801837,  0.41478688,  0.71809804, -0.80333924,
        -0.6750596 , -0.8355486 , -0.12525035, -1.3608888 ,  0.48752967,
         0.15277374,  0.1669693 , -0.46396255, -0.6256493 , -0.54297155,
        -0.12714136,  0.2301707 , -0.04148962, -0.342102  ,  0.02770735,
         0.40544727, -0.491386  ,  0.4701867 , -1.0919001 ]],
      dtype=float32)

In [35]:
weights[0]

array([[-0.0503917 , -0.09018501, -0.10317633, -0.07002152,  0.07048383,
         0.20034751,  0.03185269,  0.14860214,  0.00930676,  0.20367874,
        -0.02039749, -0.05157167,  0.2922875 ,  0.10555371,  0.02545188,
        -0.03003182, -0.19478583, -0.07915115, -0.18803139,  0.27225205,
        -0.04037363, -0.22213322,  0.15193217, -0.1770821 ,  0.17975444,
         0.17992312, -0.07199208, -0.24020289, -0.12940721, -0.21287514,
        -0.33252025,  0.08783754, -0.08330618, -0.17894925,  0.15183564,
         0.17498413,  0.13799916, -0.04276556, -0.25003728,  0.0306367 ,
        -0.08955789,  0.17244901, -0.16065481,  0.07809982,  0.10101651,
        -0.13242991,  0.19582762, -0.19676921,  0.11630262,  0.24266164,
        -0.0814794 ,  0.08935064,  0.08448912, -0.04943654,  0.13372892,
        -0.23600054,  0.23739482, -0.11731467, -0.07949226,  0.19738239,
        -0.06473543,  0.2566641 , -0.33080667,  0.16484594,  0.16337974,
         0.15103786,  0.04341464,  0.08513369,  0.0

In [24]:
aa=tf.constant([1, 2, 3, 4, 5, 6, 7,8])
bb,cc = tf.split(aa,2,0)

In [25]:
with tf.Session() as sess:
    a,b,c=sess.run([aa,bb,cc])
    

In [28]:
print(c)

[5 6 7 8]
